In [2]:
from glob import glob
import os.path as op
import nibabel as nib
import shutil
from shutil import copyfile
import pathlib
import pickle
import gzip
from glob import glob
import nibabel as nb
import numpy as np
import os

In [3]:
#Step 1: Define fMRIPrep working directory i.e., the TEMP folder
work_dir = ('/home/ttha0011/kg98/Thapa/WinnieData/4b_fMRIPrepv_20_2_1/Temp')

#Step 2: Define fMRIPrep derivatives directory
deriv_dir = ('/home/ttha0011/kg98/Thapa/WinnieData/4b_fMRIPrepv_20_2_1/derivatives/fmriprep')

#Step 3: Define subject ID
subID = ['103']

#Step 4: Rename bold registration, head motion correction, and susceptibility distortion correction work-flow
xform_rename = {'bold_reg_wf/bbreg_wf/fsl2itk_fwd/affine.txt': '_from-reference_to-T1w_mode-image_xfm.txt',
                'bold_hmc_wf/fsl2itk/mat2itk.txt': '_from-native_to-reference_mode-image_xfm.txt',
                'sdc_wf/pepolar_unwarp_wf/cphdr_warp/_warpfieldQwarp_PLUS_WARP_fixhdr.nii.gz': '_sdc_warpfieldQwarp.nii.gz'}

for sub in subID:
    sub_in_dir = op.join(work_dir, 'sub-{0}/fmriprep_wf/single_subject_{0}_wf'.format(sub))
    task_dirs = glob(op.join(sub_in_dir, 'func_preproc_task_{0}_echo_1_wf'.format("REST")))
    print(task_dirs)
    for task_dir in task_dirs:
        bb_wf_dir = op.join(task_dir, 'bold_bold_trans_wf')
        bf_dirs = sorted(glob(op.join(bb_wf_dir, '_bold_file_*')))        
        for bf_dir in bf_dirs:
            bf_dir_list = bf_dir.split('..')           
            idx = bf_dir_list.index('sub-{0}'.format(sub))
            sub_deriv_dir = op.join(deriv_dir, op.dirname('/'.join(bf_dir_list[idx:])))
            bf_filename = bf_dir_list[-1]           
            in_file = op.join(bf_dir, 'merge/vol0000_xform-00000_merged.nii.gz')
            orig_fn_list = bf_filename.split('_')      
            fn_list = orig_fn_list[:-1]
            fn_list.insert(len(fn_list),'bold.nii.gz')
            fn_list.insert(-1, 'space-native')
            fn_list.insert(-1, 'desc-partialPreproc')
            out_file = op.join(sub_deriv_dir, '_'.join(fn_list))
            copyfile(in_file, out_file)  
            #print(out_file)
            
            # Collect native-to-T1w and T1w-to-MNI transforms
            out_func_dir = op.dirname(out_file)
            #print(out_func_dir)            
            f = op.join(task_dir, 'bold_std_trans_wf', '_std_target_MNI152NLin2009cAsym.res2/bold_to_std_transform/_inputs.pklz')
            with gzip.open(f, 'rb') as fo:
                #print (fo)
                data = pickle.load(fo)
   


['/home/ttha0011/kg98/Thapa/WinnieData/4b_fMRIPrepv_20_2_1/Temp/sub-103/fmriprep_wf/single_subject_103_wf/func_preproc_task_REST_echo_1_wf']


This section extracts individual echoes from fMRIPrep's workflow directory

In [130]:
#Step 1: Define path to fMRIPrep folder
path =('/home/ttha0011/kg98/Thapa/WinnieData/4b_fMRIPrepv_20_2_1')

#Step 2: Enter subject ID
subID = ('103')

#Step 3: Enter condition i.e., DC2, dcPT1, ShamBlock, or eStim
task = ('REST')

#Step 4: Define path to anatomical folder. Need the xfm.h5 for T1w_to_MNI transformations
anatpath = os.path.join(path,'derivatives/fmriprep/sub-{}/anat'.format (subID))
     
#Step 5: Define path to functional folder. Need the exhoes and the transformation files
funcpath = os.path.join(path,'derivatives/fmriprep/sub-{}/func'.format (subID))

#Step 6: Define path to bold_hmc_wfpath folder.
bold_hmc_wfpath = os.path.join(path,'Temp/sub-{}/fmriprep_wf/single_subject_{}_wf/func_preproc_task_REST_echo_1_wf/bold_hmc_wf/fsl2itk'.format (subID,subID))

#Step 6: Define path to bold_hmc_wfpath folder.
sdc_wfpath = os.path.join(path,'Temp/sub-{}/fmriprep_wf/single_subject_{}_wf/func_preproc_task_REST_echo_1_wf/sdc_estimate_wf/pepolar_unwarp_wf/cphdr_warp'.format (subID,subID))

#Step 6: Define path to where you will copy the extracted echoes, and transformation files
ExtractedEchoesFolder = ('/home/ttha0011/kg98/Thapa/WinnieData/5_ExtractedEchoes')

#Step 7: Make directory based upon subject ID
os.makedirs(ExtractedEchoesFolder + '/sub-' + subID, exist_ok=True) 

#Step 8: Make directory based upon the condition in each subject's folder 
os.makedirs(ExtractedEchoesFolder + '/sub-' + subID + '/' + task, exist_ok=True) 

#Step 9: Define path to where you will copy the extracted echoes, and transformation files after creating necessary sub-directories
ConditionFolder = os.path.join(ExtractedEchoesFolder + '/sub-' + subID+ '/' + task. format(subID))

#/sub-103/fmriprep_wf/single_subject_103_wf/func_preproc_task_REST_echo_1_wf/bold_hmc_wf/fsl2itk'

#Step 10: Define the files to copy. These should be the same for each participant. File1 = anat folder; file2 - 7: func files
file1 = ('sub-{}_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5'.format(subID))
file2 = ('sub-{}_task-{}_echo-1_space-native_desc-partialPreproc_bold.nii.gz'.format(subID, task))
file3 = ('sub-{}_task-{}_echo-2_space-native_desc-partialPreproc_bold.nii.gz'.format(subID, task))
file4 = ('sub-{}_task-{}_echo-3_space-native_desc-partialPreproc_bold.nii.gz'.format(subID, task))
file5 = ('sub-{}_task-{}_echo-4_space-native_desc-partialPreproc_bold.nii.gz'.format(subID, task))
file6 = ('mat2itk.txt')
file7 = ('sub-{}_from-fsnative_to-T1w_mode-image_xfm.txt'.format(subID, task))
file8 = ('_warpfieldQwarp_PLUS_WARP_fixhdr.nii.gz'.format(subID, task))

#Step 10: Copy files to the right dirctory i.e., the ConditionFolder. File1 = anat folder; file2 - 7: func files
shutil.copy(anatpath + '/' + file1, ConditionFolder)
shutil.copy(funcpath + '/' + file2, ConditionFolder)
shutil.copy(funcpath + '/' + file3, ConditionFolder)
shutil.copy(funcpath + '/' + file4, ConditionFolder)
shutil.copy(funcpath + '/' + file5, ConditionFolder)
shutil.copy(bold_hmc_wfpath + '/' + file6, ConditionFolder)
shutil.copy(anatpath + '/' + file7, ConditionFolder)
shutil.copy(sdc_wfpath + '/' + file8, ConditionFolder)


dest_dir = ('/home/ttha0011/kg98/Thapa/WinnieData/5_ExtractedEchoes/sub-{}/REST/'.format(subID)) 
native_to_T1w = os.path.join(dest_dir, 'sub-103_from-fsnative_to-T1w_mode-image_xfm.txt')
rename_native_to_T1w = os.path.join(dest_dir, ('sub-{}_task-REST_from-reference_to-T1w_mode-image_xfm.txt').format(subID))
os.rename(native_to_T1w, rename_native_to_T1w)

hmc_file = os.path.join(dest_dir, 'mat2itk.txt')
rename_hmc_file = os.path.join(dest_dir, ('sub-{}_task-REST_from-native_to-reference_mode-image_xfm.txt').format(subID))
os.rename(hmc_file, rename_hmc_file)

sdc_file = os.path.join(dest_dir, '_warpfieldQwarp_PLUS_WARP_fixhdr.nii.gz')
rename_sdc_file = os.path.join(dest_dir, ('sub-{}_task-REST_sdc_warpfieldQwarp.nii.gz').format(subID))
os.rename(sdc_file, rename_sdc_file)

This section copies the extracted echoes to the Tedana folder. Use these echoes to run Tedana

In [131]:
#This script organises files to run Tedana. Note that the paths have been set in the cell above 

#Step 1: Define path to where the echoes extracted from fMRIPrep's workflow directory are located
ExtractedEchoesFolder = ('/home/ttha0011/kg98/Thapa/WinnieData/5_ExtractedEchoes/sub-{}/{}'.format (subID, task))

#Step 2: Define path to where you will copy the extracted echoes
TedanaFolder = ('/home/ttha0011/kg98/Thapa/WinnieData/6_Tedana')

#Step 3: Make directory in the TedanaFolder based upon subject ID
os.makedirs(TedanaFolder + '/sub-' + subID, exist_ok=True) 

#Step 4: Make directory based upon the condition in each subject's folder in the Tedana directory
os.makedirs(TedanaFolder + '/sub-' + subID + '/' + task, exist_ok=True) 

#Step 5: Make input (echoes files) and output (Tedana ouputs) folders in each condition's folder 
os.makedirs(TedanaFolder + '/sub-' + subID + '/' + task + '/' + 'Input', exist_ok=True) 
os.makedirs(TedanaFolder + '/sub-' + subID + '/' + task + '/' + 'Output', exist_ok=True) 

#Step 6: Define path to where you will copy the extracted echoes
EchoesFolder = os.path.join(TedanaFolder + '/sub-' + subID + '/' + task + '/' + 'Input')

#Step 7: Define echo names 
Echo1 = ('sub-{}_task-{}_echo-1_space-native_desc-partialPreproc_bold.nii.gz'.format(subID, task))
Echo2 = ('sub-{}_task-{}_echo-2_space-native_desc-partialPreproc_bold.nii.gz'.format(subID, task))
Echo3 = ('sub-{}_task-{}_echo-3_space-native_desc-partialPreproc_bold.nii.gz'.format(subID, task))
Echo4 = ('sub-{}_task-{}_echo-4_space-native_desc-partialPreproc_bold.nii.gz'.format(subID, task))

#Step 8: Copy, and paste each echo in the Tedana folder 
shutil.copy(ExtractedEchoesFolder + '/' + Echo1, EchoesFolder)
shutil.copy(ExtractedEchoesFolder + '/' + Echo2, EchoesFolder)
shutil.copy(ExtractedEchoesFolder + '/' + Echo3, EchoesFolder)
shutil.copy(ExtractedEchoesFolder + '/' + Echo4, EchoesFolder)

'/home/ttha0011/kg98/Thapa/WinnieData/6_Tedana/sub-103/REST/Input/sub-103_task-REST_echo-4_space-native_desc-partialPreproc_bold.nii.gz'

This section copies the transformations to run ANTs

In [132]:
#This script organises files to run ANTs after running Tedana

#Step 1: Define path to where the transformations extracted from fMRIPrep are located
ExtractedEchoesFolder = ('/home/ttha0011/kg98/Thapa/WinnieData/5_ExtractedEchoes/sub-{}/{}'.format (subID, task))

#Step 2: Define path to where you will copy the transformation files
TransformationsFolder = ('/home/ttha0011/kg98/Thapa/WinnieData/7_Transformations')

#Step 3: Make directory based upon subject ID in the transformation folder
os.makedirs(TransformationsFolder + '/sub-' + subID, exist_ok=True) 

#Step 4: Make directory based upon the condition in each subject's folder in the transformation folder
os.makedirs(TransformationsFolder + '/sub-' + subID + '/' + task, exist_ok=True) 

#Step 5: Define path to where you will copy the transformation files 
TransFolder = os.path.join(TransformationsFolder + '/sub-' + subID + '/' + task)

#Step 6: Define transformation files
Trans1 = ('sub-{}_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5'.format(subID))
Trans2 = ('sub-{}_task-{}_from-native_to-reference_mode-image_xfm.txt'.format(subID, task))
Trans3 = ('sub-{}_task-{}_from-reference_to-T1w_mode-image_xfm.txt'.format(subID, task))
Trans4 = ('sub-{}_task-{}_sdc_warpfieldQwarp.nii.gz'.format(subID, task))

#Step 7: Copy, and paste transformation files to the transformation folder
shutil.copy(ExtractedEchoesFolder + '/' + Trans1, TransFolder)
shutil.copy(ExtractedEchoesFolder + '/' + Trans2, TransFolder)
shutil.copy(ExtractedEchoesFolder + '/' + Trans3, TransFolder)
shutil.copy(ExtractedEchoesFolder + '/' + Trans4, TransFolder)


'/home/ttha0011/kg98/Thapa/WinnieData/7_Transformations/sub-103/REST/sub-103_task-REST_sdc_warpfieldQwarp.nii.gz'